In [1]:
import pandas as pd
import torch

In [2]:
# Importing from csvs

df_irr = pd.read_csv("../Data/CSVS/irr_full.csv")
df_pvout = pd.read_csv("../Data/CSVS/pvout_full.csv")

In [3]:
print(df_irr.shape)
print(df_pvout.shape)

(24219204, 4)
(159541752, 4)


In [4]:
df_irr.head()

,latitude,longitude,irradiance,month
0,59.97,-167.58,13.265174,1
1,59.97,-167.54,13.263229,1
2,59.97,-167.50,13.261991,1
3,59.97,-167.46,13.196024,1
4,59.97,-167.42,13.367926,1


In [5]:
df_pvout.head()

,latitude,longitude,pvo,month
0,59.979167,-167.179167,0.947,1
1,59.979167,-167.170833,0.926,1
2,59.979167,-167.162500,0.905,1
3,59.979167,-167.154167,0.884,1
4,59.979167,-167.145833,0.855,1


In [3]:
# Created rounded columns

df_pvout['lat_2dp'] = df_pvout['latitude'].round(2)
df_pvout['lon_2dp'] = df_pvout['longitude'].round(2)

In [4]:
# Group and average columns with identical rounded lats and longs

df_pvout_grouped = df_pvout.groupby(['lat_2dp', 'lon_2dp', 'month']).mean().reset_index()

In [5]:
# Merge the dfs

combined_df = pd.merge(
    df_pvout_grouped, 
    df_irr, 
    left_on=['lat_2dp', 'lon_2dp', 'month'], 
    right_on=['latitude', 'longitude', 'month'],
    how='inner'
)


In [6]:
# Drop unnecessary columns

combined_df = combined_df.drop(columns=['latitude_x', 'longitude_x', 'latitude_y', 'longitude_y'])

In [15]:
combined_df.shape

(5431476, 5)

In [16]:
combined_df.head()

,lat_2dp,lon_2dp,month,pvo,irradiance
0,18.85,-155.74,1,4.7340,194.719617
1,18.85,-155.74,2,4.9785,221.339286
2,18.85,-155.74,3,5.1175,243.341858
3,18.85,-155.74,4,5.2020,264.600658
4,18.85,-155.74,5,5.1510,273.507216


In [9]:
# Add elevation data

import googlemaps
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")
gmaps = googlemaps.Client(key=API_KEY)
BASE_URL = "https://maps.googleapis.com/maps/api/elevation/json"

In [16]:
# Batch insert elevation data

import requests
import time

unique_coords = combined_df[['lat_2dp', 'lon_2dp']].drop_duplicates().reset_index(drop=True)

def batch_coordinates(df, batch_size=256):
    latitudes = df['lat_2dp'].tolist()
    longitudes = df['lon_2dp'].tolist()
    for i in range(0, len(df), batch_size):
        yield list(zip(latitudes[i:i+batch_size], longitudes[i:i+batch_size]))

def get_elevation_batch(batch, max_retries=2):
    locations = "|".join([f"{lat},{lon}" for lat, lon in batch])
    params = {
        "locations": locations,
        "key": API_KEY
    }
    retries = 0
    while retries <= max_retries:
        try:
            response = requests.get(BASE_URL, params=params, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if data["status"] == "OK":
                    return data["results"]
                else:
                    print(f"[Error] API responded with status: {data['status']}")
                    return None
            else:
                print(f"[HTTP Error] Status code {response.status_code}")
                return None
        except Exception as e:
            print(f"[Retry {retries+1}] Error fetching data: {e}")
            retries += 1
            time.sleep(1.5 * retries)
    print("[Failed] Max retries exceeded for a batch")
    return None

elevation_lookup = {}

total_batches = (len(unique_coords) // 256) + 1
batch_counter = 0

start_time = time.time()

for batch in batch_coordinates(unique_coords):
    batch_counter += 1
    results = get_elevation_batch(batch)
    
    batch_keys = [tuple(coords) for coords in batch]

    if results:
        for coords, res in zip(batch_keys, results):
            elevation_lookup[coords] = res['elevation']
    else:
        for coords in batch_keys:
            elevation_lookup[coords] = None 

    # Progress print every 10 batches
    if batch_counter % 10 == 0 or batch_counter == total_batches:
        elapsed = time.time() - start_time
        print(f"Processed {batch_counter}/{total_batches} batches "
              f"({(batch_counter/total_batches)*100:.2f}%), "
              f"Elapsed time: {elapsed/60:.1f} min")
    
    time.sleep(0.2) 

combined_df['elevation'] = combined_df.apply(
    lambda row: elevation_lookup.get((row['lat_2dp'], row['lon_2dp']), None), axis=1
)


Exception ignored in: <function tqdm.__del__ at 0x1137cf4c0>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ml-0451/lib/python3.9/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/opt/anaconda3/envs/ml-0451/lib/python3.9/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


Processed 10/1769 batches (0.57%), Elapsed time: 0.1 min
Processed 20/1769 batches (1.13%), Elapsed time: 0.1 min
Processed 30/1769 batches (1.70%), Elapsed time: 0.2 min
Processed 40/1769 batches (2.26%), Elapsed time: 0.3 min
Processed 50/1769 batches (2.83%), Elapsed time: 0.3 min
Processed 60/1769 batches (3.39%), Elapsed time: 0.4 min
Processed 70/1769 batches (3.96%), Elapsed time: 0.5 min
Processed 80/1769 batches (4.52%), Elapsed time: 0.5 min
Processed 90/1769 batches (5.09%), Elapsed time: 0.6 min
Processed 100/1769 batches (5.65%), Elapsed time: 0.6 min
Processed 110/1769 batches (6.22%), Elapsed time: 0.7 min
Processed 120/1769 batches (6.78%), Elapsed time: 0.8 min
Processed 130/1769 batches (7.35%), Elapsed time: 0.8 min
Processed 140/1769 batches (7.91%), Elapsed time: 0.9 min
Processed 150/1769 batches (8.48%), Elapsed time: 1.0 min
Processed 160/1769 batches (9.04%), Elapsed time: 1.0 min
Processed 170/1769 batches (9.61%), Elapsed time: 1.1 min
Processed 180/1769 batc

Exception ignored in: <function tqdm.__del__ at 0x1137cf4c0>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ml-0451/lib/python3.9/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/opt/anaconda3/envs/ml-0451/lib/python3.9/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


Processed 770/1769 batches (43.53%), Elapsed time: 5.0 min
Processed 780/1769 batches (44.09%), Elapsed time: 5.0 min
Processed 790/1769 batches (44.66%), Elapsed time: 5.1 min
Processed 800/1769 batches (45.22%), Elapsed time: 5.2 min
Processed 810/1769 batches (45.79%), Elapsed time: 5.2 min
Processed 820/1769 batches (46.35%), Elapsed time: 5.3 min
Processed 830/1769 batches (46.92%), Elapsed time: 5.3 min
Processed 840/1769 batches (47.48%), Elapsed time: 5.4 min
Processed 850/1769 batches (48.05%), Elapsed time: 5.5 min
Processed 860/1769 batches (48.62%), Elapsed time: 5.5 min
Processed 870/1769 batches (49.18%), Elapsed time: 5.6 min
Processed 880/1769 batches (49.75%), Elapsed time: 5.7 min
Processed 890/1769 batches (50.31%), Elapsed time: 5.7 min
Processed 900/1769 batches (50.88%), Elapsed time: 5.8 min
Processed 910/1769 batches (51.44%), Elapsed time: 5.9 min
Processed 920/1769 batches (52.01%), Elapsed time: 5.9 min
Processed 930/1769 batches (52.57%), Elapsed time: 6.0 m

In [17]:
combined_df.head()

,lat_2dp,lon_2dp,month,pvo,irradiance,elevation
0,18.85,-155.74,1,4.7340,194.719617,-2015.588257
1,18.85,-155.74,2,4.9785,221.339286,-2015.588257
2,18.85,-155.74,3,5.1175,243.341858,-2015.588257
3,18.85,-155.74,4,5.2020,264.600658,-2015.588257
4,18.85,-155.74,5,5.1510,273.507216,-2015.588257


In [18]:
combined_df.shape

(5431476, 6)

In [20]:
combined_df.to_csv('../Data/CSVS/csvs_combined.csv', index=False, encoding='utf-8')